**Разбор инстумента docker-compose**

Позиционируется как инстумент, который следующая ступень абстрации над docker - инстумент который позволяет организовать взаимодейсвие контейнеров между собой.

# YAML

Язык разметки, с помощью которого задается поведение docker-compose.

YAML позволяет описывать структуры данных которые состоят из списков и словарей:
- каждый элемент списка начинается с символа `-`;
- каждый новый ключ в словаре задатся так `<ключ>:`;
- шаблоны (якоря) - возможность создать ссылку на некоторую сущность и потом использовать её в произвольном месте структуры;
- вложенность структуры формируется отступами.

Далее на примерах станет понятнее.

В python существует библиотека yaml, которая позволяет пребразовывать yaml разметку в соответсвующие структуры данных python.

In [1]:
import yaml

В следующем примере создается ключ словарь с ключами `names`, `ages` под которыми скрываются соответсвующие списки.

In [2]:
yamls_str = \
"""
names:
    - peter
    - olga
ages:
    - 22
    - 18
"""

yaml.full_load(yamls_str)

{'names': ['peter', 'olga'], 'ages': [22, 18]}

Заметим, что между `<ключ>:` и `<значение>` обязательно должен быть пробел. Вот, для сравнения, правильно созданный словарь под ключом `postgres` и ошибочно созданный под ключом `clickhouse`.

In [3]:
yamls_str = \
"""
postgres:
    user: postgres_app_user
    password: postgres_app_password
    host: postgres_host
    port: 5432
clickhouse:
    host:clickhouse_host
    user:clickhouse_app_user
    db:clickhouse_app_db
    password:clickhouse_app_password
"""

yaml.full_load(yamls_str)

{'postgres': {'user': 'postgres_app_user',
  'password': 'postgres_app_password',
  'host': 'postgres_host',
  'port': 5432},
 'clickhouse': 'host:clickhouse_host user:clickhouse_app_user db:clickhouse_app_db password:clickhouse_app_password'}

Инетестно то, что для того, что по умолчанию yaml игнорирует все переносы на новую строку. Для того, чтобы справиться с этим исполюзуется:
- `|` после имени ключа заставил yaml "видеть" перевод строки;
- `>` после имени ключа воткнет перенос строки в конец занчения.

Так в примене ниже `test1` и `test2` с точки срения программы читающей yaml не отличаются. А вот `test3` и `test4` получают в некоторых местах служебный `\n`.

In [4]:
yamls_str = \
"""
test1: peter olga
test2:
    peter
    olga
test3: |
    peter
    olga
test4: >
    peter olga
"""

yaml.full_load(yamls_str)

{'test1': 'peter olga',
 'test2': 'peter olga',
 'test3': 'peter\nolga\n',
 'test4': 'peter olga\n'}

Ну и для примера покажем как сформировать список словарей:

In [5]:
yamls_str = \
"""
- name: peter
  age: 22
- name: olga
  age: 18
"""

yaml.full_load(yamls_str)

[{'name': 'peter', 'age': 22}, {'name': 'olga', 'age': 18}]

Якоря создаются следующим образом:
- В сущности на которую ссылаются задают `&<обозначение ссылки>`;
- Когда эту сущность надо вставить куда-то используется синтаксис `<<: *<обозначение ссылки>`.

В примере далее были описаны свойства junior-a некоторой компании, а затем созданы две сушности которым были переданы свойсва этих junior-ов. 

In [6]:
yamls_str = \
"""
junior:
    &junior
    position: junior
    salary: 55000

Peter:
    <<: *junior
Olga:
    <<: *junior
"""

yaml.full_load(yamls_str)

{'junior': {'position': 'junior', 'salary': 55000},
 'Peter': {'position': 'junior', 'salary': 55000},
 'Olga': {'position': 'junior', 'salary': 55000}}

# Команды `docker-compose`

Тут будут рассмотрены самые полезные случаи для работы с коммандой `docker-compose`. Для примера используется `docker-compose.yaml` приложенный в той-же папке, что и этот notebook.

### `up` - поднять приложение

Команда `up` используется для того, чтобы поднять приложение спользующее `docker-compose`. Выполняется обязательно в той папке в которой лежит `yaml` описывающий приложение.

Опции:

- `d` - запустит в фоновом режиме - терминал останеться под управлением пользователя.

Так, следующий пример показывает, что до вызова `docker-comporse up` в docker нет не контейнеров ни вольюмов, а после, все это появляется.

In [12]:
%%bash
echo '=======запущенные контейнеры========='
docker ps
echo '===========доступные volume=========='
docker volume ls

echo '==========запускаю приложение==========='
docker-compose up -d &> /dev/null

echo '=======запущенные контейнеры========='
docker ps --format '{{.Names}}'
echo '===========доступные volume=========='
docker volume ls

docker-compose down -v &> /dev/null

=======запущенные контейнеры=========
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
===========доступные volume==========
DRIVER    VOLUME NAME
==========запускаю приложение===========
=======запущенные контейнеры=========
docker_compose-db-1
===========доступные volume==========
DRIVER    VOLUME NAME
local     docker_compose_ex_vol


### `down` - положить приложение

Опять же требуется вызывать из папки в которой лежит `yaml` описывающий приложение.

Опции:
- `v` - удалит все volume созданные при поднятии этого приложения.

In [13]:
%%bash
docker-compose up -d &> /dev/null
docker-compose down &> /dev/null

echo '============без опции -v============'
docker volume ls
docker volume rm docker_compose_ex_vol > /dev/null 2>&1


docker-compose up -d &> /dev/null
docker-compose down -v &> /dev/null
echo '============опция -v============'
docker volume ls

============без опции -v============
DRIVER    VOLUME NAME
local     docker_compose_ex_vol
============опция -v============
DRIVER    VOLUME NAME


# Создание `docker-compose` файла

Здается с помощью описанного выше языка `yaml`. Далее будем обсуждать ключи которые могут быть использованы и для чего они могут быть использованы.

### Ключ `services`

Скрывает под собой описание контейнеров, которые будут использоваться при развертывании приложения. Каждый ключ в словале под `seriveces` создает новый контенер, ключи задаются произвольно. То есть выглядеть это должно прмерно так:

```
services:
  <контейнер1>:
    <инструкции>
  <контейнер2>:
    <инструкии>
  ...
```

Далее рассмотрим различные интструкции которые может содржать каждый из контейнеров:

##### **Базовые интсрукции**

Есть ряд интсрукции которые просто воспроизводят некторые комманды обычного `docker`. Не считаю, что они заслуживают отдельного разбора (пока не столкнулся с проблемами), потому просто покажу соответсвие с коммандами docker.

| docker-compose.yaml  | docker  |
|---|---|
| `image: <образ>`  |  `docker run <образ>` |
| `container_name: <имя контейнера> `| `docker run --name <имя контейнера>`|
| `volumes:` <br> `- <volume/путь на хосте 1>:<путь в контейнере1>` <br> `- <volume/путь на хосте 2>:<путь в контейнере2>` <br> ...| `docker run \`<br>`-v <путь на хосте/volume 1>:<путь в контейнере1>\` <br> `-v <путь на хосте/volume 2>:<путь в контейнере2>\` <br> ...|
| `environment:`<br>`<имя переменной1>: <значение1>`<br> `<имя переменной2>: <значение2>` <br> ... | `docker run \` <br> `-e <имя переменной1>=<значение переменной1>` <br> `-e <имя переменной2>=<значение переменной2>` <br> ...|
| `networks:` <br> `- <сеть 1>` <br> `- <сеть 2>` <br> ... | `docker run` <br> `--net <сеть 1>` <br> `--net <сеть 2>` <br> ...|
| `ports: ` <br> `<порт на хосте1>:<порт в контейнере1>` <br> `<порт на хосте2>:<порт в контейнере2>` <br> ...| `docker run` <br> `-p <порт на хосте1>:<порт в контейнере1>` <br> `-p <порт на хосте2>:<порт в контейнере2>` <br> ...|

##### Сборка образа - инструкция `build`

Соответсвует комманде docker build. Соберет образ и запустит на его основе контейнер. заметим, что при опускании приложения, docker-compose остановит и удалит контейнер, но, не образ, поэтому удаление образа (в случае необходимости) ложится на админа.

In [14]:
%%bash
cd build_example
docker-compose up -d &> /dev/null
echo '=====показываю что появился новый образ====='
docker images | grep build_example_my_small_ubuntu
echo '=====а на его основании контейнер====='
docker ps -a --format '{{.Names}}'
docker-compose down &> /dev/null
echo '=====опустил приложение, но образ то остался====='
docker images | grep build_example_my_small_ubuntu
docker rmi build_example_my_small_ubuntu &> /dev/null

=====показываю что появился новый образ=====
build_example_my_small_ubuntu   latest    098799dc601d   9 days ago      77.8MB
=====а на его основании контейнер=====
build_example-my_small_ubuntu-1
=====опустил приложение, но образ то остался=====
build_example_my_small_ubuntu   latest    098799dc601d   9 days ago      77.8MB


##### Интерактивный режим и подключение `tty`

Делаются интрукциями:

```
tty: true
stdin_open: true
```

Далее пример. Там поднимаются два контейнера на основе ubuntu один:
- `with_tty` использует названные инструкции;
- `no_tty` не использует названные инструкции.

В итоге при вызовер `docker ps -a` выявляюется обра образа, а при `docker ps` только образ `with_tty`, что говорит о том, что образ `no_tty` завершает свою работу.

In [15]:
%%bash

cd it_option
docker-compose up -d &> /dev/null
sleep 5
echo "=====docker ps -a====="
docker ps -a --format '{{.Names}}'
echo "=====docker ps====="
docker ps --format '{{.Names}}'
docker-compose down &> /dev/null

=====docker ps -a=====
no_tty
with_tty
=====docker ps=====
with_tty


##### Перезапуск контейнера - интструкция `restart`

В случае, если некоторая программа при определенных обстоятельсвах будет вылетать с ошбкой её приходится перезапускать. В docker-compose для того предусмотренна инструация `restart`. Она, похоже, может принимать множество значений, но на сегоняшний день известна только `always`, которая приведет к тому, что контейнер будет перезапускаться пока не запустится.

Образ приведенный в папке `restart` сделан таким образом, чтобы программа останавливась всегда, когда файл `check_file` сорержит число меньшеее 5, но приращает число файле на 1. Если же программа видит число большее либо равное 5-ти в контейнере, то это приводит к тому, что она зависает в бесконечном цикле.

Запуская `docker-compose.yaml` с таким контейнером внутри и опцией `restart: true` получаем, что записанное в файле число дорастает ровно до 5.

In [20]:
%%bash

cd restart
echo "1" > check_file

docker-compose up -d &> /dev/null
sleep 10

docker-compose down &> /dev/null
docker rmi example_python &> /dev/null

cat check_file

5

##### Проверка работоспособности контейнера - инструкция `healthcheck`

Иногоа бывает так, что контейнер может провериться, по поводу того насколько правильно он запущен. Для того, чтобы вызвать команду проверки используется интсрукция `healthy`.

`healthy` имеет некоторые настройки, вот некоторые из них:
- `test` - задает команду которая будет исопльзована для проверки контейнера;
- `interval`;
- `timeout`;
- `retries`.

Вот, например, как инструкция может быть использована в postgresql:

```
...
healthcheck: 
    test: ["CMD-SHELL", "pg_isready", "-U", "docker_app"]
...
```

Так в примере, представленном ниже, разворачивается два контейнера на основе postgresql (на остальные контейнеры в рамках этого примера можно не обращать внимания). Один с этой опицией, другой без. При отображении их через `dokcer ps` у одного в `STATUS` есть пририска `(health: starting)` у второго - нет. 

In [16]:
%%bash
cd postgres_example
docker-compose up -d &> /dev/null
docker ps --format '{{.Names}}'
docker-compose down -v &> /dev/null

postgres_example-ubuntu3-1
postgres_example-ubuntu1-1
postgres_example-db1-1
postgres_example-db2-1


##### Зависимость от других сревисов - `depends_on`

Если требуется что-то делать в зависимости от другого сервиса в этом `docker-compose.yaml` то делается это так:

```
...
depends_on:
    <имя обуславливающего сервиса>:
        <инструкции>
...
```

Например, если требуется запускать контейнер, только в том случае, если другой сервис `healthy`, то это будет записано так:

```
...
depends_on:
    <имя обуславливающего сервиса>:
        condition: service_healthy
...
```

На данный момент известно о следующих возможностях этой инструкции:

- `condition: service_healthy` - проверить проведена ли для обулавливающего контейнера проверка `healthcheck`;
- `condition: service_started` - проверить поднят ли обуславливающий контейнер.

Для примера представим "docker-compose.yaml" расположенный в папке "postgres_example". В нем: 
- сервисы "ubuntu1" и "ubuntu2" поднимаются только в случае "здоровья" сервисов "db1" и "db2" соотсветсвенно - в результате поднимается только "ubuntu1" потому как проверка `healthcheck` предусмотрена только для "db1";
- сервисы "ubuntu3" и "ubuntu4" поднимаются только в тех случах если подняты "ubuntu1" и "ubuntu2" соотственно - так как на прошлом шаге "unbuntu2" не поднялся не поднимется и "ubuntu4".

In [17]:
%%bash
cd postgres_example
docker-compose up -d &> /dev/null
docker ps --format '{{.Names}}'
docker-compose down -v &> /dev/null

postgres_example-ubuntu3-1
postgres_example-ubuntu1-1
postgres_example-db2-1
postgres_example-db1-1


##### Реплики контейнера - интсрукция `deploy->replicas`

Для интсрукции `deploy`, наверняка будет больше подинтсрукций. Но на сегдняшний день известно только об `replicas`.

`deploy->replicas` позволяет задать число копий поднимаемого контейнера.

Так использование последовательности инструкций:
```
deploy:
  replicas: 3
```

Приводит к тому, что будет поднято три реплики сервиса для которого это записано.

Так в примере ниже я одним махом поднимаю 3 контейнера ubuntu:

In [18]:
%%bash
cd replicas
docker-compose up -d &> /dev/null
docker ps -a --format '{{.Names}}'
docker-compose down &> /dev/null

replicas-my_ubuntu-1
replicas-my_ubuntu-3
replicas-my_ubuntu-2


##### Команды при запуске - инструкция `command`

Можно указать команду, которая будет выполнена при запуске контейнера. Так "docker-file.yaml" в папке "command" содержить строку:<br>
`command: bash -c "echo \"hello world\" > test_file; bash"`

Эта строка в контейре создаёт файл "test_file" наличие и содержание, котого мы проверяем в примере ниже:

In [1]:
%%bash
cd command
docker-compose up -d &> /dev/null
docker exec temp_ubuntu cat test_file
docker-compose down &> /dev/null

hello world


Интерестно, то что если написть `command` так:

`command echo "hello world" > test_file`

Контейнер завершает работу. Видимо это связано с тем, что последняя комманда должна бесконечно ожидать ввода, чтобы все работало - именно такую функцию выполняет комманда `bash`.

### Ключ `volumes`

Задает volumes.

Каждый вложенный ключ создаст volume.

```
volumes:
    <volume1>:
        name: <volume name>
    <volume2>:
    ...
```

У каждого volume можно задать поле `name` (а можно и не задавать) которое укажет имя volume при поднятии приложения. Так, в примере ниже, из папки volume_example создается volume с именем `example_name`.

In [10]:
%%bash
cd volume_example
docker-compose up -d &> /dev/null

echo "=====список volume====="
docker volume ls

docker-compose down -v &> /dev/null

=====список volume=====
DRIVER    VOLUME NAME
local     example_name


### Ключ `networks`

Задает сети импользуемые в приложении.

Каждый вложенный ключ создаст сеть.

```
networks:
    <net1>:
        name: <network name>
    <net2>:
    ...
```

Ключ `name` задает имя сети и не является обязательным. На в папке `network-example` лежит `yaml` файл, который описывает сеть с именем `example_name`.

In [11]:
%%bash
cd network_example
docker-compose up -d &> /dev/null
docker network ls
docker-compose down -v &> /dev/null

NETWORK ID     NAME           DRIVER    SCOPE
75034612bd8f   bridge         bridge    local
14cb969f28d4   example_name   bridge    local
f8b2503d0640   host           host      local
b1d7e6bda275   none           null      local


# Детали

### Сеть по умолчанию

Любое приложение запущенное через `docker-compose`, в случае отсудсвия указанных сетей, создает себе сеть с названием по типу `<имя папки>_default`. Так в примере далее показано, что в списке, кроме базовых сетей, появляется `docker_compose_default`.

In [12]:
%%bash
docker-compose up -d &> /dev/null
echo "=====созданные сети====="
docker network ls
docker-compose down -v &> /dev/null

=====созданные сети=====
NETWORK ID     NAME                     DRIVER    SCOPE
75034612bd8f   bridge                   bridge    local
205bf876f99f   docker_compose_default   bridge    local
f8b2503d0640   host                     host      local
b1d7e6bda275   none                     null      local


### Название по умолчанию

volumes/сети, по умолчанию, получают некоторые названия по типу `<название папки>_<название ключа>`. Так, в следующем примере, в папке `default_namimg` указаны volume `ex_vol` и сеть `ex_net`, но для них не указано ключа `name`.

In [13]:
%%bash
cd default_naming
docker-compose up -d &> /dev/null
echo '=====volumes====='
docker volume ls
echo '=====networks====='
docker network ls
docker-compose down -v &> /dev/null

=====volumes=====
DRIVER    VOLUME NAME
local     default_naming_ex_vol
=====networks=====
NETWORK ID     NAME                    DRIVER    SCOPE
75034612bd8f   bridge                  bridge    local
4599a4f61b14   default_naming_ex_net   bridge    local
f8b2503d0640   host                    host      local
b1d7e6bda275   none                    null      local


### volume/сеть не создаются?

Убедитесь, что они указаны под ключами volumes/networks в каком-либо из контейнеров. В противном случае они не создаются. 

В следующем примере из папки `vol_net_missed` разворачивается приложение. И хотя в соответсвующем `docker-compose.yaml` заданы volume `ex_vol` и сеть `ex_net`, при запуске приложения создается только безимянный volume (видимо создаваемый postgres по умолчанию) и сеть которая всегда создается `docker-compose` по умолчанию `vol_net_missed_default`.

Примеры удачного создания сетей/volumes представлены выше.

In [14]:
%%bash
cd vol_net_missed
docker-compose up -d &> /dev/null
echo '====volumes====='
docker volume ls
echo '=====networks====='
docker network ls
docker-compose down -v &> /dev/null

====volumes=====
DRIVER    VOLUME NAME
local     48e548f659d0e05ba4a981e4b17e9b2b1835ca62aeb0cbfa08974d23d7772469
=====networks=====
NETWORK ID     NAME                     DRIVER    SCOPE
75034612bd8f   bridge                   bridge    local
f8b2503d0640   host                     host      local
b1d7e6bda275   none                     null      local
bcef30a7a541   vol_net_missed_default   bridge    local


### Монтирование директорий из директирии запуска

Иногда требуется сослатся на папку из которой производится запуск `docker-compose`. Для того можно использовать `${PWD}/<дирректория>` или `./<дирректория>`

Когда мы, например, в `docker` монтировали файл/папку, через bind mount мы писали что-то вроде:
```
docker run --rm\
    -v $(pwd)/<название файла/папки>:<путь в контейнере>
```

Аналогичная запись в `docker-compose.yaml`:

```
services:
  <название сервиса>:
    volumes:
      - ${PWD}/<название файла/папки>:<путь в контейнере>
```

 или

```
services:
  <название сервиса>:
    volumes:
      - ./<название файла/папки>:<путь в контейнере>
```

Пример далее, заодно покажем как пользоваться bind mount в docker-compose.

docker-compose файл лежит в папке pwd_example. В этой же папке создается файл `test_file`, который потом указано монтировать в файле docker-compose.yaml, так `- ${PWD}/test_file:/test_file1` и как `- ./test_file:test_file2`. 

Затем в файлы в файловой системе контейнера вносятся изменения.

После закрытия контейнера, все изменения в файле остаются в исходном файле на хосте.

In [5]:
%%bash

cd pwd_example
echo "Сообщение с хоста" > test_file
echo "=====Исходный файл====="
cat test_file

docker-compose up -d &> /dev/null
docker exec test_cont bash -c "echo 'Сообщение из контейнера 1' >> test_file1"
docker exec test_cont bash -c "echo 'Сообщение из контейнера 2' >> test_file2"
docker-compose down &> /dev/null

echo "=====Измененный файл====="
cat test_file

=====Исходный файл=====
Сообщение с хоста
=====Измененный файл=====
Сообщение с хоста
Сообщение из контейнера 1
Сообщение из контейнера 2


### Обновление docker-compose приложения

Если вдруг, в `docker-compose.yaml` были внесены некоторые изменения, то не обязательно опускать приложение - можно его просто запустить наново, docker-compose подхватит, что это то же самое приложение (видимо по папке запуска) и обновит его.

В следующем примере, я сначала запускаю docker-compose с приложением в котором единтсвенный контейнер имеет имя `first_name`, затем подменяю `docker-compose.yaml` на тот, который содержит контейнер названный `second_name`.

Я хочу показать то, что несмотря на то что запуска два и немного разных контейнер то остается один и меняет имя в соовтесвии с актуальным `docker-compose.yaml`

In [19]:
%%bash
cd refresh
cat first.yaml > docker-compose.yaml
docker-compose up -d &> /dev/null
echo "=====Первый запуск====="
docker ps -a --format '{{.Names}}'
cat second.yaml > docker-compose.yaml
docker-compose up -d &> /dev/null
echo "=====Второй запуск====="
docker ps -a --format '{{.Names}}'
docker-compose down &> /dev/null

=====Первый запуск=====
first_name
=====Второй запуск=====
second_name
